Election de 2022 premier tour

In [160]:
import numpy as np
import pandas as pd



import os
os.listdir()

['migrations',
 'election2017.ipynb',
 'pop-2007.ipynb',
 '.DS_Store',
 'populations',
 'population_2007_2023.xlsx',
 'alembic.ini',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'scripts',
 'db',
 'chomages',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'elections']

#### Importer la database

Import des datasets.

In [161]:
df = pd.read_excel('./populations/pop-2007.xlsx')

Visualise toutes les colones

In [162]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Départements,Unnamed: 1,Ensemble,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Hommes,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Femmes,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,NaN,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus,Total,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus,Total,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus,Total
1,01,Ain,153719,147284,163003,68837,41534,574377,78672,74187,81771,33496,15625,283751,75047,73097,81232,35341,25909,290626
2,02,Aisne,141079,131304,152341,67757,45339,537820,72302,65638,76034,31402,16208,261584,68777,65666,76307,36355,29131,276236
3,03,Allier,71446,74685,97923,56218,42842,343114,36753,37614,48399,25918,15787,164471,34693,37071,49524,30300,27055,178643
4,04,Alpes-de-Haute-Provence,35905,32707,45416,25895,16144,156067,18655,15991,22354,12322,6247,75569,17250,16716,23062,13573,9897,80498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,95,Val-d'Oise,336059,328308,323309,112904,60141,1160721,172309,160395,156871,54232,21198,565005,163750,167913,166438,58672,38943,595716
97,971,Guadeloupe,122977,98651,110972,45406,22578,400584,63118,44730,51025,20686,8766,188325,59859,53921,59947,24720,13812,212259
98,972,Martinique,113813,97129,113223,48495,25070,397730,57391,44191,51680,22116,9592,184970,56422,52938,61543,26379,15478,212760
99,973,Guyane,94722,63937,41967,9220,3185,213031,47623,30538,21381,4720,1284,105546,47099,33399,20586,4500,1901,107485


In [163]:
def split_and_explode_population(df):
    """
    Sépare le dataframe source en hommes / femmes,
    puis transforme chaque sous-dataframe en format long
    """

    # --- 1. Séparation hommes / femmes ---
    df_homme = pd.concat([df.iloc[:, :2], df.iloc[:, 8:14]], axis=1)
    df_femme = pd.concat([df.iloc[:, :2], df.iloc[:, 14:20]], axis=1)

    return df_homme, df_femme

In [164]:
df_homme, df_femme = split_and_explode_population(df)

df_homme.head()

,Départements,Unnamed: 1,Hommes,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,0 à 19 ans,20 à 39 ans,40 à 59 ans,60 à 74 ans,75 ans et plus,Total
1,01,Ain,78672,74187,81771,33496,15625,283751
2,02,Aisne,72302,65638,76034,31402,16208,261584
3,03,Allier,36753,37614,48399,25918,15787,164471
4,04,Alpes-de-Haute-Provence,18655,15991,22354,12322,6247,75569


In [165]:
import pandas as pd
import math

def explode_population_dataframe(df, sexe, annee, debug=True):
    """
    Transforme un dataframe large (par tranche d'âge)
    en dataframe long (1 ligne = 1 tranche)
    """
    lignes = []

    tranche_map = {
        2: "0 à 19 ans",
        3: "20 à 39 ans",
        4: "40 à 59 ans",
        5: "60 à 74 ans",
        6: "75 ans et plus"
    }

    if debug:
        print("=== DÉBUT TRAITEMENT DATAFRAME ===")
        print(f"Sexe traité : {sexe}")
        print(f"Nombre de lignes source : {len(df)}")
        print("-" * 50)

    for idx, row in df.iterrows():

        code_raw = row.iloc[0]
        nom_dept = row.iloc[1]

        # 🚫 SKIP ligne d'en-tête ou invalide
        if pd.isna(code_raw) or not str(code_raw).isdigit():
            if debug:
                print(f"[LIGNE {idx}] ⏭️ ignorée (ligne d'en-tête ou invalide)")
            continue

        code_dept = str(code_raw).zfill(2)

        if debug:
            print(f"[LIGNE {idx}] Département {code_dept} - {nom_dept}")

        for col_idx, tranche in tranche_map.items():
            population = row.iloc[col_idx]

            # sécurité supplémentaire
            if pd.isna(population):
                population = 0

            if debug:
                print(
                    f"  → Tranche: {tranche:<15} | "
                    f"Population: {population}"
                )

            lignes.append({
                "code_dept": code_dept,
                "nom_dept": nom_dept,
                "sexe": sexe,
                "tranche_age": tranche,
                "population": int(population),
                "annee": annee
            })

        if debug:
            print("-" * 50)

    df_result = pd.DataFrame(lignes)

    if debug:
        print("=== FIN TRAITEMENT ===")
        print(f"Nombre total de lignes générées : {len(df_result)}")
        print("Aperçu du résultat :")
        print(df_result.head())

    return df_result


In [166]:
explode_population_dataframe(df_homme, "H", 2007, False).head()


,code_dept,nom_dept,sexe,tranche_age,population,annee
0,01,Ain,H,0 à 19 ans,78672,2007
1,01,Ain,H,20 à 39 ans,74187,2007
2,01,Ain,H,40 à 59 ans,81771,2007
3,01,Ain,H,60 à 74 ans,33496,2007
4,01,Ain,H,75 ans et plus,15625,2007


In [167]:
explode_population_dataframe(df_femme, "H", 2007, False).head()

,code_dept,nom_dept,sexe,tranche_age,population,annee
0,01,Ain,H,0 à 19 ans,75047,2007
1,01,Ain,H,20 à 39 ans,73097,2007
2,01,Ain,H,40 à 59 ans,81232,2007
3,01,Ain,H,60 à 74 ans,35341,2007
4,01,Ain,H,75 ans et plus,25909,2007


In [168]:
import pandas as pd

df_population_all = []

for annee in range(2007, 2023):
    print(f"\n=== Traitement population {annee} ===")

    # 1. Lecture du fichier Excel
    df = pd.read_excel(f"./populations/pop-{annee}.xlsx")

    # Affichage rapide des premières lignes
    # print("Aperçu du fichier brut :")
    # print(df.head(2))
    
    # 2. Séparation hommes / femmes
    df_homme, df_femme = split_and_explode_population(df)

    # 3. Explosion (format long) avec debug activé
    df_homme_long = explode_population_dataframe(
        df_homme,
        sexe="H",
        annee=annee,
        debug=False
    )

    df_femme_long = explode_population_dataframe(
        df_femme,
        sexe="F",
        annee=annee,
        debug=False
    )

    # 4. Concaténer hommes + femmes pour cette année
    df_population_all.append(pd.concat([df_homme_long, df_femme_long], ignore_index=True))

# 5. Concaténation finale sur toutes les années
df_population = pd.concat(df_population_all, ignore_index=True)


print("\n=== Aperçu final population ===")
print(df_population.head(15))
print("\nNombre total de lignes :", len(df_population))


=== Traitement population 2007 ===

=== Traitement population 2008 ===

=== Traitement population 2009 ===

=== Traitement population 2010 ===

=== Traitement population 2011 ===

=== Traitement population 2012 ===

=== Traitement population 2013 ===

=== Traitement population 2014 ===

=== Traitement population 2015 ===

=== Traitement population 2016 ===

=== Traitement population 2017 ===

=== Traitement population 2018 ===

=== Traitement population 2019 ===

=== Traitement population 2020 ===

=== Traitement population 2021 ===

=== Traitement population 2022 ===

=== Aperçu final population ===
   code_dept nom_dept sexe     tranche_age  population  annee
0         01      Ain    H      0 à 19 ans       78672   2007
1         01      Ain    H     20 à 39 ans       74187   2007
2         01      Ain    H     40 à 59 ans       81771   2007
3         01      Ain    H     60 à 74 ans       33496   2007
4         01      Ain    H  75 ans et plus       15625   2007
5         02    Ais

In [ ]:
#df_population.to_excel("./exports/xlsx/population_2007_2023.xlsx", index=False)

In [ ]:
from db import SessionLocal
from models import Sexe, TrancheAge


session = SessionLocal()



In [181]:
def seed_sexe(session):
    codes = ["ENSEMBLE", "H", "F"]
    for code in codes:
        if not session.query(Sexe).filter_by(code=code).first():
            session.add(Sexe(code=code))

In [182]:
def seed_tranche_age(session):
    tranches = [
        (0, 19),
        (20, 39),
        (40, 59),
        (60, 74), 
        (75, None),
    ]

    for age_min, age_max in tranches:
        exists = session.query(TrancheAge).filter_by(
            age_min=age_min,
            age_max=age_max
        ).first()
        if not exists:
            session.add(
                TrancheAge(age_min=age_min, age_max=age_max)
            )

In [183]:
def run():
    session = SessionLocal()
    try:
        seed_sexe(session)
        seed_tranche_age(session)
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [ ]:
if __name__ == "__main__":
    run()